# EMT Syngen LoadStep - 9th Order DCIM versus VBR - Controller Tests

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os
import subprocess

#%matplotlib widget

PEAK1PH_TO_RMS3PH=np.sqrt(3./2.)

root_path = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')

path_exec = root_path + '/build/dpsim/examples/cxx/'

finalTime = 30.0

timeStepDCIM = 100e-6
timeStepVBR = 500e-6

loadStepEventTime = 5.0
loadFactor = 1.1

## Run simulation and read results of DCIM without controllers

In [ ]:
name_exec = 'EMT_SynchronGenerator9OrderDCIM_LoadStep_TurbineGovernor_Exciter'

model_name = name_exec

sim = subprocess.Popen([path_exec+name_exec, '--name', model_name, '--timestep', str(timeStepDCIM), '--duration', str(finalTime), '--option', 'TIMESTEPEVENT='+str(loadStepEventTime), '--option', 'LOADFACTOR='+str(loadFactor)], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(sim.communicate()[0].decode())

path = 'logs/' + model_name + '/'
dpsim_result_file = path  + model_name + '.csv'
ts_dpsim_emt_dcim = read_timeseries_csv(dpsim_result_file)

## Run simulation and read results of VBR without controllers

In [ ]:
name_exec = 'EMT_SynchronGenerator9OrderVBR_LoadStep_TurbineGovernor_Exciter'

model_name = name_exec

sim = subprocess.Popen([path_exec+name_exec, '--name', model_name, '--timestep', str(timeStepVBR), '--duration', str(finalTime), '--option', 'TIMESTEPEVENT='+str(loadStepEventTime), '--option', 'LOADFACTOR='+str(loadFactor)], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(sim.communicate()[0].decode())

path = 'logs/' + model_name + '/'
dpsim_result_file = path  + model_name + '.csv'
ts_dpsim_emt_vbr = read_timeseries_csv(dpsim_result_file)

## Generator bus voltages

In [ ]:
plt.figure(figsize=(12,8))
name = 'v1_0'
plt.plot(ts_dpsim_emt_dcim[name].time, PEAK1PH_TO_RMS3PH*ts_dpsim_emt_dcim[name].values, label='dcim')
plt.plot(ts_dpsim_emt_vbr[name].time, PEAK1PH_TO_RMS3PH*ts_dpsim_emt_vbr[name].values, label='vbr', linestyle='--')
plt.legend()
plt.show()

## Generator current

In [ ]:
plt.figure(figsize=(12,8))
name = 'i_gen_0'
plt.plot(ts_dpsim_emt_dcim[name].time, ts_dpsim_emt_dcim[name].values, label='dcim')
plt.plot(ts_dpsim_emt_vbr[name].time, ts_dpsim_emt_vbr[name].values, label='vbr', linestyle='--')
plt.legend()
plt.show()

## Generator omega

In [ ]:
plt.figure(figsize=(12,8))
name = 'wr_gen'
plt.plot(ts_dpsim_emt_dcim[name].time, ts_dpsim_emt_dcim[name].values, label='dcim')
plt.plot(ts_dpsim_emt_vbr[name].time, ts_dpsim_emt_vbr[name].values, label='vbr', linestyle='--')
plt.legend()
plt.show()

## Assertion

In [ ]:
name = 'wr_gen'
rmse = ts_dpsim_emt_dcim[name].rmse(ts_dpsim_emt_dcim[name], ts_dpsim_emt_vbr[name].interpolate(timeStepDCIM))
print(rmse)
assert(rmse < 4e-4)

## Run simulation and read results of DCIM with governor

In [ ]:
name_exec = 'EMT_SynchronGenerator9OrderDCIM_LoadStep_TurbineGovernor_Exciter'

model_name = name_exec

sim = subprocess.Popen([path_exec+name_exec, '--name', model_name, '--timestep', str(timeStepDCIM), '--duration', str(finalTime), '--option', 'WITHGOVERNOR=true', '--option', 'TIMESTEPEVENT='+str(loadStepEventTime), '--option', 'LOADFACTOR='+str(loadFactor)], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(sim.communicate()[0].decode())

path = 'logs/' + model_name + '/'
dpsim_result_file = path  + model_name + '.csv'
ts_dpsim_emt_dcim = read_timeseries_csv(dpsim_result_file)

## Run simulation and read results of VBR with governor

In [ ]:
name_exec = 'EMT_SynchronGenerator9OrderVBR_LoadStep_TurbineGovernor_Exciter'

model_name = name_exec

sim = subprocess.Popen([path_exec+name_exec, '--name', model_name, '--timestep', str(timeStepVBR), '--duration', str(finalTime), '--option', 'WITHGOVERNOR=true', '--option', 'TIMESTEPEVENT='+str(loadStepEventTime), '--option', 'LOADFACTOR='+str(loadFactor)], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(sim.communicate()[0].decode())

path = 'logs/' + model_name + '/'
dpsim_result_file = path  + model_name + '.csv'
ts_dpsim_emt_vbr = read_timeseries_csv(dpsim_result_file)

## Generator bus voltages

In [ ]:
plt.figure(figsize=(12,8))
name = 'v1_0'
plt.plot(ts_dpsim_emt_dcim[name].time, PEAK1PH_TO_RMS3PH*ts_dpsim_emt_dcim[name].values, label='dcim - with governor')
plt.plot(ts_dpsim_emt_vbr[name].time, PEAK1PH_TO_RMS3PH*ts_dpsim_emt_vbr[name].values, label='vbr - with governor', linestyle='--')
plt.legend()
plt.show()

## Generator current

In [ ]:
plt.figure(figsize=(12,8))
name = 'i_gen_0'
plt.plot(ts_dpsim_emt_dcim[name].time, ts_dpsim_emt_dcim[name].values, label='dcim - with governor')
plt.plot(ts_dpsim_emt_vbr[name].time, ts_dpsim_emt_vbr[name].values, label='vbr - with governor', linestyle='--')
plt.legend()
plt.show()

## Generator omega

In [ ]:
plt.figure(figsize=(12,8))
name = 'wr_gen'
plt.plot(ts_dpsim_emt_dcim[name].time, ts_dpsim_emt_dcim[name].values, label='dcim - with governor')
plt.plot(ts_dpsim_emt_vbr[name].time, ts_dpsim_emt_vbr[name].values, label='vbr - with governor', linestyle='--')
plt.legend()
plt.show()

## Assertion

In [ ]:
name = 'wr_gen'
rmse = ts_dpsim_emt_dcim[name].rmse(ts_dpsim_emt_dcim[name], ts_dpsim_emt_vbr[name].interpolate(timeStepDCIM))
print(rmse)
assert(rmse < 2.1e-5)

## Run simulation and read results of DCIM with governor and exciter

In [ ]:
name_exec = 'EMT_SynchronGenerator9OrderDCIM_LoadStep_TurbineGovernor_Exciter'

model_name = name_exec

sim = subprocess.Popen([path_exec+name_exec, '--name', model_name, '--timestep', str(timeStepDCIM), '--duration', str(finalTime), '--option', 'WITHGOVERNOR=true', '--option', 'WITHEXCITER=true', '--option', 'TIMESTEPEVENT='+str(loadStepEventTime), '--option', 'LOADFACTOR='+str(loadFactor)], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(sim.communicate()[0].decode())

path = 'logs/' + model_name + '/'
dpsim_result_file = path  + model_name + '.csv'
ts_dpsim_emt_dcim = read_timeseries_csv(dpsim_result_file)

## Run simulation and read results of VBR with governor and exciter

In [ ]:
name_exec = 'EMT_SynchronGenerator9OrderVBR_LoadStep_TurbineGovernor_Exciter'

model_name = name_exec

sim = subprocess.Popen([path_exec+name_exec, '--name', model_name, '--timestep', str(timeStepVBR), '--duration', str(finalTime), '--option', 'WITHGOVERNOR=true', '--option', 'WITHEXCITER=true', '--option', 'TIMESTEPEVENT='+str(loadStepEventTime), '--option', 'LOADFACTOR='+str(loadFactor)], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(sim.communicate()[0].decode())

path = 'logs/' + model_name + '/'
dpsim_result_file = path  + model_name + '.csv'
ts_dpsim_emt_vbr = read_timeseries_csv(dpsim_result_file)

## Generator bus voltages

In [ ]:
plt.figure(figsize=(12,8))
name = 'v1_0'
plt.plot(ts_dpsim_emt_dcim[name].time, PEAK1PH_TO_RMS3PH*ts_dpsim_emt_dcim[name].values, label='dcim - with governor and exciter')
plt.plot(ts_dpsim_emt_vbr[name].time, PEAK1PH_TO_RMS3PH*ts_dpsim_emt_vbr[name].values, label='vbr - with governor and exciter', linestyle='--')
plt.legend()
plt.show()

## Generator current

In [ ]:
plt.figure(figsize=(12,8))
name = 'i_gen_0'
plt.plot(ts_dpsim_emt_dcim[name].time, ts_dpsim_emt_dcim[name].values, label='dcim - with governor and exciter')
plt.plot(ts_dpsim_emt_vbr[name].time, ts_dpsim_emt_vbr[name].values, label='vbr - with governor and exciter', linestyle='--')
plt.legend()
plt.show()

## Generator omega

In [ ]:
plt.figure(figsize=(12,8))
name = 'wr_gen'
plt.plot(ts_dpsim_emt_dcim[name].time, ts_dpsim_emt_dcim[name].values, label='dcim - with governor and exciter')
plt.plot(ts_dpsim_emt_vbr[name].time, ts_dpsim_emt_vbr[name].values, label='vbr - with governor and exciter', linestyle='--')
plt.legend()
plt.show()

## Assertion

In [ ]:
name = 'wr_gen'
rmse = ts_dpsim_emt_dcim[name].rmse(ts_dpsim_emt_dcim[name], ts_dpsim_emt_vbr[name].interpolate(timeStepDCIM))
print(rmse)
assert(rmse < 8e-6)